In [2]:
# Mounting colab with drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Imports

import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time


### Extracting Flats/Apartments


```
- Your_Project_Directory
  - Data
    - City
      - Flats
      - Societies
      - Residential
      - Independent House
```

In [19]:
# Need to change as per your requirement - city name
# Taking value of city as 'chandigarh'
City = 'Mumbai'

In [4]:
# User Agent
# Headers set like below:
# User Agent
headers = {
    'authority': 'www.mobilebricks.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'dnt': '1',
    'pragma': 'no-cache',
    'referer': f'https://www.mobilebricks.com/flats-in-{City}-ffid-page',
    'sec-ch-ua': '"Chromium";v="107", "Not;A=Brand";v="8"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/527.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
}

In [5]:
# If folder structures are in already created no need to run it.

import os

# Define the path to your project directory
project_dir = '/content/drive/MyDrive/pro/Real estate/'

# Define the subdirectories
subdirectories = ['Data', f'Data/{City}', f'Data/{City}/Flats', f'Data/{City}/Societies', f'Data/{City}/Residential', f'Data/{City}/Independent House']

# Create the directory structure
for subdir in subdirectories:
    dir_path = os.path.join(project_dir, subdir)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print(f"Created directory: {dir_path}")
    else:
        print(f"Directory already exists: {dir_path}")

# Now, your directory structure is created.


Directory already exists: /content/drive/MyDrive/pro/Real estate/Data
Directory already exists: /content/drive/MyDrive/pro/Real estate/Data/Mumbai
Directory already exists: /content/drive/MyDrive/pro/Real estate/Data/Mumbai/Flats
Directory already exists: /content/drive/MyDrive/pro/Real estate/Data/Mumbai/Societies
Directory already exists: /content/drive/MyDrive/pro/Real estate/Data/Mumbai/Residential
Directory already exists: /content/drive/MyDrive/pro/Real estate/Data/Mumbai/Independent House


In [11]:
import re
import json

In [9]:
url = 'https://www.magicbricks.com/flats-in-mumbai-for-sale-pppfs/page-1'
page = requests.get(url, headers=headers)
pageSoup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
for soup in pageSoup.select_one('div.mb-srp__left').select_one('div.mb-srp__list'):
    print(soup.select_one('h2.mb-srp__card--title').text.strip())
    # Extract the text content from the script tags
    script_texts = [script.text for script in soup.select('script')]
    # Search for the pattern in the extracted text content
    match = re.search(r'\{.*\}', ''.join(script_texts))
    if match:
        print(type(match.group()))

3 BHK Flat for Sale in Goregaon West, Mumbai
<class 'str'>


In [10]:
for soup in pageSoup.select_one('div.mb-srp__left').select_one('div.mb-srp__list'):
  json_data = soup.find('script', type='application/ld+json').string.strip()
  # Parse the JSON data
  try:
    data = json.loads(json_data)

    print("Name:", data.get('name', 'N/A'))
    print("Type:", data.get('@type', 'N/A'))
    print("Seller Name:", data.get('potentialAction', {}).get('seller', {}).get('name', 'N/A'))
    print("Latitude:", data.get('geo', {}).get('latitude', 'N/A'))
    print("Longitude:", data.get('geo', {}).get('longitude', 'N/A'))
    print("Locality:", data.get('address', {}).get('addressLocality', 'N/A'))
    print("Region:", data.get('address', {}).get('addressRegion', 'N/A'))
    print("Country:", data.get('address', {}).get('addressCountry', 'N/A'))
    url_in = data.get('url', 'N/A')
    if url_in != 'N/A':
      page_in = requests.get(url_in, headers=headers)
      pageSoup_in = BeautifulSoup(page_in.content, 'html.parser')

      div = pageSoup_in.find('div', class_='mb-ldp__dtls__body__list')
      main_text = f"{div.contents[0].strip()} {div.find('span', class_='mb-ldp__dtls__body__list--units').find('span').text}"

      summary = pageSoup_in.find_all('li', class_='mb-ldp__dtls__body__summary--item')

      summary_text = [su.get_text(strip=True) for su in summary]
      result = ', '.join(summary_text)

      print(result)
      items = pageSoup_in.find_all('div', class_='mb-ldp__dtls__body__summary--item isPremium')

      # Extract the text from each div and join them into a comma-separated string
      text_list = [item.get_text(strip=True) for item in items]
      result = ', '.join(text_list)

      print(result)
      print(pageSoup_in.find('div',class_='mb-ldp__dtls__price').text.strip())

      amenities = pageSoup_in.find_all('li', class_='mb-ldp__amenities__list--item')

      amenities_text = [amenity.get_text(strip=True) for amenity in amenities]
      result = ', '.join(amenities_text)

      print(result)
      data_dict = {}
      for item in pageSoup_in.find_all('li',class_='mb-ldp__dtls__body__list--item'):
        label = item.find('div', class_='mb-ldp__dtls__body__list--label').get_text(strip=True)
        value_div = item.find('div', class_='mb-ldp__dtls__body__list--value')

        link = value_div.find('a')
        if link:
            value = link.get_text(strip=True)
        else:
            value = value_div.get_text(strip=True)

        if label == 'Carpet Area' or label == 'Super Built-up Area':
          size_text = item.find('div', class_='mb-ldp__dtls__body__list--size').get_text(strip=True)
          value = main_text + ',' + size_text
        data_dict[label] = value
      print(data_dict)

  except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")

NameError: name 'json' is not defined

In [12]:
for soup in pageSoup.select_one('div.mb-srp__left').select_one('div.mb-srp__list'):
  json_data = soup.find('script', type='application/ld+json').string.strip()
  # Parse the JSON data
  try:
    data = json.loads(json_data)

    print("Name:", data.get('name', 'N/A'))
    print("Type:", data.get('@type', 'N/A'))
    print("Seller Name:", data.get('potentialAction', {}).get('seller', {}).get('name', 'N/A'))
    print("Latitude:", data.get('geo', {}).get('latitude', 'N/A'))
    print("Longitude:", data.get('geo', {}).get('longitude', 'N/A'))
    print("Locality:", data.get('address', {}).get('addressLocality', 'N/A'))
    print("Region:", data.get('address', {}).get('addressRegion', 'N/A'))
    print("Country:", data.get('address', {}).get('addressCountry', 'N/A'))
    url_in = data.get('url', 'N/A')
    if url_in != 'N/A':
      page_in = requests.get(url_in, headers=headers)
      pageSoup_in = BeautifulSoup(page_in.content, 'html.parser')
      # print(pageSoup_in.find('div',class_='mb-ldp__dtls__body__list').text.strip())
      # Find the div with the specified class
      div = pageSoup_in.find('div', class_='mb-ldp__dtls__body__list')

      # # Extract the main text (1072)
      # main_text = div.contents[0].strip()

      # # Extract the unit (sqft)
      # unit_text = div.find('span', class_='mb-ldp__dtls__body__list--units').find('span').text

      # Combine the main text and unit text
      main_text = f"{div.contents[0].strip()} {div.find('span', class_='mb-ldp__dtls__body__list--units').find('span').text}"

      # Print the result
      # print(result)
      print(pageSoup_in.find('ul',class_='mb-ldp__dtls__body__summary').text.strip())
      # print(pageSoup_in.find('div',class_='mb-ldp__dtls__body__summary--item ico-beds').text.strip())
      # print(pageSoup_in.find('div',class_='mb-ldp__dtls__body__summary--item ico-baths').text.strip())
      # print(pageSoup_in.find('div',class_='mb-ldp__dtls__body__summary--item ico-balconies').text.strip())
      # print(pageSoup_in.find('div',class_='mb-ldp__dtls__body__summary--item ico-furnished').text.strip())
      # print(pageSoup_in.find('div',class_='mb-ldp__dtls__body__summary--right').text.strip())
      # Find all divs with class "mb-ldp__dtls__body__summary--item isPremium"
      items = pageSoup_in.find_all('div', class_='mb-ldp__dtls__body__summary--item isPremium')

      # Extract the text from each div and join them into a comma-separated string
      text_list = [item.get_text(strip=True) for item in items]
      result = ', '.join(text_list)

      # Print the result
      print(result)
      print(pageSoup_in.find('div',class_='mb-ldp__dtls__price').text.strip())
      # print(pageSoup_in.find('ul',{'class':'mb-ldp__lux'}).text.strip())
      # Find all span elements with class "amenities__item__text"
      amenities = pageSoup_in.find_all('li', class_='mb-ldp__amenities__list--item')

      # Extract the text from each span and join them into a comma-separated string
      amenities_text = [amenity.get_text(strip=True) for amenity in amenities]
      result = ', '.join(amenities_text)

      # Print the result
      print(result)
      data_dict = {}
      for item in pageSoup_in.find_all('li',class_='mb-ldp__dtls__body__list--item'):
        label = item.find('div', class_='mb-ldp__dtls__body__list--label').get_text(strip=True)
        value_div = item.find('div', class_='mb-ldp__dtls__body__list--value')

        # If there's a link in the value
        link = value_div.find('a')
        if link:
            value = link.get_text(strip=True)
        else:
            value = value_div.get_text(strip=True)

        if label == 'Carpet Area' or label == 'Super Built-up Area':
          # main_text = item.find('div', class_='mb-ldp__dtls__body__list').get_text(strip=True)
          size_text = item.find('div', class_='mb-ldp__dtls__body__list--size').get_text(strip=True)
          value = main_text + ',' + size_text
        data_dict[label] = value
      # item = pageSoup_in.find('li', class_='mb-ldp__dtls__body__list--item')
      # data_dict['Super Built-up Area'] = item.find('div', class_='mb-ldp__dtls__body__list').get_text(strip=True) + item.find('div', class_='mb-ldp__dtls__body__list--size').get_text(strip=True)
      print(data_dict)

  except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")

Name: 2 BHK Flat  for Sale in  NG Royal Park, Kanjurmarg East, Mumbai
Type: Apartment
Seller Name: kanayalal
Latitude: 19.12021420298
Longitude: 72.9299633704782
Locality: Kanjurmarg East
Region: Mumbai
Country: IN
2Beds2BathsSemi-Furnished

₹1.50 Cr
Power Back Up, Lift, Club House, Swimming Pool, Gymnasium, Park
{'Carpet Area': '625 sqft,₹24,000/sqft', 'Developer': 'RNA NG Builders', 'Project': 'NG Royal Park', 'Floor': '5 (Out of 15 Floors)', 'Transaction type': 'Resale', 'Status': 'Ready to Move', 'Additional Rooms': '1 Store Room', 'Facing': 'East', 'Lifts': '3'}


In [16]:
url_in

'https://www.magicbricks.com/propertyDetails/2-BHK-953-Sq-ft-Multistorey-Apartment-FOR-Sale-Kanjurmarg-East-in-Mumbai&id=4d423731313133363135'

In [ ]:
# pageSoup_in

In [ ]:
# for soup in pageSoup.select_one('div.mb-srp__left').select_one('div.mb-srp__list'):
#   json_data = soup.find('script', type='application/ld+json').string.strip()
#   # Parse the JSON data
#   try:
#     data = json.loads(json_data)

#     # Access and print all fields in the JSON data
#     # print("Context:", data.get('@context', 'N/A'))
#     print("Type:", data.get('@type', 'N/A'))
#     # print("ID:", data.get('@id', 'N/A'))
#     print("URL:", data.get('url', 'N/A'))
#     print("Number of Rooms:", data.get('numberOfRooms', 'N/A'))
#     print("Image URL:", data.get('image', 'N/A'))
#     print("Name:", data.get('name', 'N/A'))
#     # print("Geo Type:", data.get('geo', {}).get('@type', 'N/A'))
#     print("Latitude:", data.get('geo', {}).get('latitude', 'N/A'))
#     print("Longitude:", data.get('geo', {}).get('longitude', 'N/A'))
#     # print("Action Type:", data.get('potentialAction', {}).get('@type', 'N/A'))
#     # print("Seller Type:", data.get('potentialAction', {}).get('seller', {}).get('@type', 'N/A'))
#     print("Seller Name:", data.get('potentialAction', {}).get('seller', {}).get('name', 'N/A'))
#     # print("Address Type:", data.get('address', {}).get('@type', 'N/A'))
#     print("Locality:", data.get('address', {}).get('addressLocality', 'N/A'))
#     print("Region:", data.get('address', {}).get('addressRegion', 'N/A'))
#     print("Country:", data.get('address', {}).get('addressCountry', 'N/A'))

#   except json.JSONDecodeError as e:
#     print(f"Error parsing JSON: {e}")


Type: Apartment
URL: https://www.magicbricks.com/propertyDetails/3-BHK-1072-Sq-ft-Multistorey-Apartment-FOR-Sale-Goregaon-West-in-Mumbai&id=4d423731393639323937
Number of Rooms: 3
Image URL: https://img.staticmb.com/mbphoto/property/cropped_images/2024/Apr/30/Photo_h180_w240/71969297_1_PropertyImage670-5736003633787_180_240.jpg
Name: 3 BHK Flat  for Sale in  Wadhwa Anmol Tower, Goregaon West, Mumbai
Latitude: 19.1703700678522
Longitude: 72.8424722723454
Seller Name: Krunal Pandya
Locality: Goregaon West
Region: Mumbai
Country: IN


In [25]:
city = 'mumbai'
for i in range(2):
  url = f'https://www.magicbricks.com/flats-in-{city}-for-sale-pppfs/page-{i}'
  page = requests.get(url, headers=headers)
  pageSoup = BeautifulSoup(page.content, 'html.parser')
  for soup in pageSoup.select_one('div.mb-srp__left').select_one('div.mb-srp__list'):
    json_data = soup.find('script', type='application/ld+json').string.strip()
    # Parse the JSON data
    try:
      data = json.loads(json_data)

      print("Name:", data.get('name', 'N/A'))
      print("Type:", data.get('@type', 'N/A'))
      print("Seller Name:", data.get('potentialAction', {}).get('seller', {}).get('name', 'N/A'))
      print("Latitude:", data.get('geo', {}).get('latitude', 'N/A'))
      print("Longitude:", data.get('geo', {}).get('longitude', 'N/A'))
      print("Locality:", data.get('address', {}).get('addressLocality', 'N/A'))
      print("Region:", data.get('address', {}).get('addressRegion', 'N/A'))
      print("Country:", data.get('address', {}).get('addressCountry', 'N/A'))
      url_in = data.get('url', 'N/A')
      if url_in != 'N/A':
        page_in = requests.get(url_in, headers=headers)
        pageSoup_in = BeautifulSoup(page_in.content, 'html.parser')

        summary = pageSoup_in.find_all('li', class_='mb-ldp__dtls__body__summary--item')

        summary_text = [su.get_text(strip=True) for su in summary]
        result = ', '.join(summary_text)

        print(result)
        items = pageSoup_in.find_all('div', class_='mb-ldp__dtls__body__summary--item isPremium')

        # Extract the text from each div and join them into a comma-separated string
        text_list = [item.get_text(strip=True) for item in items]
        result = ', '.join(text_list)

        print(result)
        print(pageSoup_in.find('div',class_='mb-ldp__dtls__price').text.strip())

        amenities = pageSoup_in.find_all('li', class_='mb-ldp__amenities__list--item')

        amenities_text = [amenity.get_text(strip=True) for amenity in amenities]
        result = ', '.join(amenities_text)

        print(result)
        data_dict = {}
        div = pageSoup_in.find('div', class_='mb-ldp__dtls__body__list')
        main_text = f"{div.contents[0].strip()} {div.find('span', class_='mb-ldp__dtls__body__list--units').find('span').text}"
        for item in pageSoup_in.find_all('li',class_='mb-ldp__dtls__body__list--item'):
          label = item.find('div', class_='mb-ldp__dtls__body__list--label').get_text(strip=True)
          value_div = item.find('div', class_='mb-ldp__dtls__body__list--value')

          link = value_div.find('a')
          if link:
              value = link.get_text(strip=True)
          else:
              value = value_div.get_text(strip=True)

          if label == 'Carpet Area' or label == 'Super Built-up Area':
            size_text = item.find('div', class_='mb-ldp__dtls__body__list--size').get_text(strip=True)
            value = main_text + ',' + size_text
          data_dict[label] = value
        print(data_dict)

    except json.JSONDecodeError as e:
      print(f"Error parsing JSON: {e}")

Name: 2 BHK Flat  for Sale in  Ghatkopar West, Mumbai
Type: Apartment
Seller Name: Ayush Shah
Latitude: 19.0908063
Longitude: 72.9076669
Locality: Ghatkopar West
Region: Mumbai
Country: IN
2Beds, 2Baths, Furnished

₹1.20 Cr

{'Super Built-up Area': '754 sqft,₹15,915/sqft', 'Floor': '2 (Out of 5 Floors)', 'Transaction type': 'Resale', 'Status': 'Ready to Move', 'Furnished Status': 'Furnished'}
Name: 2 BHK Flat  for Sale in  NG Royal Park, Kanjurmarg East, Mumbai
Type: Apartment
Seller Name: kanayalal
Latitude: 19.12021420298
Longitude: 72.9299633704782
Locality: Kanjurmarg East
Region: Mumbai
Country: IN
2Beds, 2Baths, Semi-Furnished

₹1.50 Cr
Power Back Up, Lift, Club House, Swimming Pool, Gymnasium, Park
{'Carpet Area': '625 sqft,₹24,000/sqft', 'Developer': 'RNA NG Builders', 'Project': 'NG Royal Park', 'Floor': '5 (Out of 15 Floors)', 'Transaction type': 'Resale', 'Status': 'Ready to Move', 'Additional Rooms': '1 Store Room', 'Facing': 'East', 'Lifts': '3'}


In [22]:
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}

In [32]:
# Put start page number and end page number.

# Page number to start extraction data
start = int(input("Enter page number where you got error in last run.\nEnter page number to start from:")) # Starting Page

# End Page number- you can change is for start i am taking 10pages at a time,
# as IPs are gettig block after some time
end = start+3

pageNumber = start
req=0

flats = pd.DataFrame()

try :
    while pageNumber < end:
        i=1
        url = 'https://www.magicbricks.com/flats-in-{City}-for-sale-pppfs/page-{pageNumber}'.format(City=city,pageNumber=pageNumber)
        page = requests.get(url, headers=headers)
        pageSoup = BeautifulSoup(page.content, 'html.parser')
        req+=1
        for soup in pageSoup.select_one('div.mb-srp__left').select_one('div.mb-srp__list'):
            json_data = soup.find('script', type='application/ld+json').string.strip()
            # Extract property name and property sub-name
            try:
                data = json.loads(json_data)

                property_name = data.get('name', 'N/A')
                seller_name = data.get('potentialAction', {}).get('seller', {}).get('name', 'N/A')
                latitude = data.get('geo', {}).get('latitude', 'N/A')
                longitude = data.get('geo', {}).get('longitude', 'N/A')
                region = data.get('address', {}).get('addressRegion', 'N/A')
                country = data.get('address', {}).get('addressCountry', 'N/A')
                # Area
                try:
                    area = data.get('address', {}).get('addressLocality', 'N/A')
                except:
                    area =''
                # Type
                try:
                    Type = data.get('@type', 'N/A')
                except:
                    Type = ''
                # Extract link
                url_in = data.get('url', 'N/A')
            except:
                continue
            # Detail Page
            if url_in != 'N/A':
              page_in = requests.get(url_in, headers=headers)
              pageSoup_in = BeautifulSoup(page_in.content, 'html.parser')
            req += 1
            try:
                #price Range
                price = pageSoup_in.find('div',class_='mb-ldp__dtls__price').text.strip()
            except:
                price = ''

            # Configuration
            try:
                summary = pageSoup_in.find_all('li', class_='mb-ldp__dtls__body__summary--item')
                if len(summary) == 0:
                    summary = pageSoup_in.find('div',class_='mb-ldp__dtls__body__summary--left mb-ldp__dtls__body__summary--dflex').find_all('div', class_='mb-ldp__dtls__body__summary--item')

                summary_text = [su.get_text(strip=True) for su in summary]
                config = ', '.join(summary_text)
            except:
                config = ''


            # Premium Items

            try:
                items = pageSoup_in.find_all('div', class_='mb-ldp__dtls__body__summary--item isPremium')

                # Extract the text from each div and join them into a comma-separated string
                text_list = [item.get_text(strip=True) for item in items]
                premium = ', '.join(text_list)
            except:
                premium = ''
            # Amenities
            try:
                amenities = pageSoup_in.find_all('li', class_='mb-ldp__amenities__list--item')

                amenities_text = [amenity.get_text(strip=True) for amenity in amenities]
                amenit = ', '.join(amenities_text)
            except:
                amenit = ''

            data_dict = {}
            div = pageSoup_in.find('div', class_='mb-ldp__dtls__body__list')
            main_text = f"{div.contents[0].strip()} {div.find('span', class_='mb-ldp__dtls__body__list--units').find('span').text}"
            for item in pageSoup_in.find_all('li',class_='mb-ldp__dtls__body__list--item'):
              label = item.find('div', class_='mb-ldp__dtls__body__list--label').get_text(strip=True)
              value_div = item.find('div', class_='mb-ldp__dtls__body__list--value')

              link = value_div.find('a')
              if link:
                  value = link.get_text(strip=True)
              else:
                  value = value_div.get_text(strip=True)

              if label == 'Carpet Area' or label == 'Super Built-up Area':
                size_text = item.find('div', class_='mb-ldp__dtls__body__list--size').get_text(strip=True)
                value = main_text + ',' + size_text
              data_dict[label] = value

            # Create a dictionary with the given variables
            property_data = {
            'property_name': property_name,
            'link': url_in,
            'seller_name': seller_name,
            'latitude': latitude,
            'longitude': longitude,
            'region': region,
            'country': country,
            'config': config,
            'premium': premium,
            'amenities': amenit,
            'price': price,
            'area': area,
            'Type': Type,
            'data_dict': data_dict
        }


            temp_df = pd.DataFrame.from_records([property_data])
            # print(temp_df)
            flats = pd.concat([flats, temp_df], ignore_index=True)
            i += 1
            # if os.path.isfile(csv_file):
            # # Append DataFrame to the existing file without header
            #     temp_df.to_csv(csv_file, mode='a', header=False, index=False)
            # else:
            #     # Write DataFrame to the file with header
            #     temp_df.to_csv(csv_file, mode='a', header=True, index=False)

            if req % 4==0:
                time.sleep(10)
            if req % 15 == 0:
                time.sleep(50)
        print(f'{pageNumber} -> {i}')
        pageNumber += 1

except AttributeError as e:
    print(e)
    print("----------------")
    print("""Your IP might have blocked. Delete Runitme and reconnect again with updating start page number.\n
            You would see in output above like 1 -> 15\ and so 1 is page number and 15 is data items extracted.""")
csv_file_path = f"/content/drive/MyDrive/pro/Real estate/Data/bhilai/Flats/flats_{City}_data-page-{start}-{pageNumber-1}.csv"

# This file will be new every time if start page will chnage, but still taking here mode as append
if os.path.isfile(csv_file_path):
# Append DataFrame to the existing file without header
  flats.to_csv(csv_file_path, mode='a', header=False, index=False)
else:
# Write DataFrame to the file with header - first time write
  flats.to_csv(csv_file_path, mode='a', header=True, index=False)


Enter page number where you got error in last run.
Enter page number to start from:1
'NoneType' object has no attribute 'select_one'
----------------
Your IP might have blocked. Delete Runitme and reconnect again with updating start page number.

            You would see in output above like 1 -> 15\ and so 1 is page number and 15 is data items extracted.


### If getting errors

* Solution for colab
```
Go to menu bar:
Runtime -> Disconnect and Delete runtime -> Reconnect again.
```

In [ ]:
# Function to combine multiple csv file is one file.

def combine_csv_files(folder_path, combined_file_path):
    combined_data = pd.DataFrame()  # Create an empty DataFrame to hold the combined data

    # Iterate through all CSV files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            print('file_path')
            # Read the data from the current CSV file
            df = pd.read_csv(file_path)

            # Append the data to the combined DataFrame
            combined_data = combined_data.append(df, ignore_index=True)

            # Delete the original CSV file
            os.remove(file_path)

    # Save the combined data to a new CSV file
    combined_data.to_csv(combined_file_path, index=False)

# Example usage:

# Replace with the actual folder path
folder_path = '/content/drive/MyDrive/DSMP/Case Studies/Real estate/flats_appartment'

# Replace with the desired combined file path
combined_file_path = '/content/drive/MyDrive/DSMP/Case Studies/Real estate/flats_appartment/flats.csv'

combine_csv_files(folder_path, combined_file_path)


file_path


<ipython-input-7-26a7d1242761>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(df, ignore_index=True)
<ipython-input-7-26a7d1242761>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(df, ignore_index=True)


file_path
